<a href="https://colab.research.google.com/github/Saikiran4545/DataScience_Project/blob/main/trading_24hr2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ccxt
import json
import time

class TradingBot:
    def __init__(self, initial_budget, symbol):
        print("Creating TradingBot instance...")
        self.budget = initial_budget
        self.symbol = symbol
        self.buy_order_placed = False
        self.load_api_credentials()

        self.kucoin_exchange = ccxt.kucoin({
            'apiKey': self.api_key,
            'secret': self.api_secret,
            'password': self.api_passphrase,
            'enableRateLimit': True,
        })

    def load_api_credentials(self):
        with open('credentials.json') as f:
            credentials = json.load(f)

        self.api_key = credentials['API_KEY']
        self.api_secret = credentials['API_SECRET']
        self.api_passphrase = credentials['API_PASSPHRASE']

    def get_24hr_high(self):
        try:
            ticker = self.kucoin_exchange.fetch_ticker(self.symbol)
            high_24hr = ticker['high']
            print(f"24-hour High for {self.symbol}: {high_24hr}")
            return high_24hr
        except ccxt.NetworkError as e:
            print(f"Network error fetching 24-hour high for {self.symbol}: {e}")
            return None
        except ccxt.ExchangeError as e:
            if 'too many requests' in str(e).lower():
                print("Too many requests. Throttling...")
                time.sleep(5)
                return self.get_24hr_high()
            else:
                print(f"Exchange error fetching 24-hour high for {self.symbol}: {e}")
                return None
        except Exception as e:
            print(f"Error fetching 24-hour high for {self.symbol}: {e}")
            return None

    def place_order(self, order_type, price, quantity):
        try:
            if order_type == "Buy":
                order = self.kucoin_exchange.create_limit_buy_order(self.symbol, quantity, price)
            elif order_type == "Sell":
                order = self.kucoin_exchange.create_limit_sell_order(self.symbol, quantity, price)

            order_id = order['id']
            print(f"{order_type} order placed - Order ID: {order_id}")
            return order_id
        except ccxt.NetworkError as e:
            print(f"Network error during order placement: {e}")
        except ccxt.ExchangeError as e:
            print(f"Exchange error during order placement: {e}")
        except Exception as e:
            print(f"Error during order placement: {e}")
        return None

    def cancel_order(self, order_id):
        try:
            self.kucoin_exchange.cancel_order(order_id)
            print(f"Order canceled - Order ID: {order_id}")
        except ccxt.NetworkError as e:
            print(f"Network error during order cancellation: {e}")
        except ccxt.ExchangeError as e:
            print(f"Exchange error during order cancellation: {e}")
        except Exception as e:
            print(f"Error during order cancellation: {e}")

    def execute_strategy(self):
        print("Executing strategy...")
        try:
            high_24hr = self.get_24hr_high()

            if high_24hr is not None:
                buy_price_threshold = 0.92 * high_24hr  # Adjusted to below 8%

                current_price = self.kucoin_exchange.fetch_ticker(self.symbol)['last']

                print(f"Buy Price Threshold: {buy_price_threshold}, Current Price: {current_price}")

                open_buy_orders = self.kucoin_exchange.fetch_open_orders(self.symbol, params={'side': 'buy'})
                open_sell_orders = self.kucoin_exchange.fetch_open_orders(self.symbol, params={'side': 'sell'})

                # Check if the budget is not already allocated and no buy order placed
                if not open_buy_orders and not self.buy_order_placed and self.budget > 0:
                    if current_price < buy_price_threshold:
                        buy_quantity = self.budget / current_price
                        order_id = self.place_order("Buy", current_price, buy_quantity)

                        if order_id:
                            self.budget -= current_price * buy_quantity
                            self.buy_order_placed = True
                            print("Buy order placed.")
                        else:
                            print("Failed to place buy order.")

                # Check if a buy order has been placed
                elif open_buy_orders:
                    # Check if the 24-hour high has changed significantly
                    new_high_24hr = self.get_24hr_high()
                    if new_high_24hr and abs(new_high_24hr - high_24hr) > 0.02 * high_24hr:
                        # Cancel the existing buy order
                        self.cancel_order(open_buy_orders[0]['id'])
                        print("Buy order canceled due to significant 24-hour high change.")

                        # Place a new buy order based on the updated 24-hour high
                        buy_price_threshold = 0.92 * new_high_24hr
                        if current_price < buy_price_threshold and self.budget > 0:
                            buy_quantity = self.budget / current_price
                            order_id = self.place_order("Buy", current_price, buy_quantity)

                            if order_id:
                                self.budget -= current_price * buy_quantity
                                self.buy_order_placed = True
                                print("New buy order placed.")
                            else:
                                print("Failed to place new buy order.")
                    else:
                        print("No significant 24-hour high change. Keeping the existing buy order.")

                # Check if a sell order should be placed
                if not open_sell_orders and self.buy_order_placed:
                    # Place a corresponding sell order at a fixed profit of 3%
                    fixed_sell_price = 1.03 * current_price
                    sell_quantity = buy_quantity  # Assuming the sell quantity is the same as the buy quantity
                    order_id = self.place_order("Sell", fixed_sell_price, sell_quantity)

                    if order_id:
                        self.buy_order_placed = False
                        print("Sell order placed.")
                    else:
                        print("Failed to place sell order.")

        except Exception as e:
            print(f"Error in executing strategy: {e}")

    def run(self):
        print("Running TradingBot...")
        try:
            while True:
                self.execute_strategy()
                time.sleep(10)

        except KeyboardInterrupt:
            print("TradingBot stopped.")

# Example usage:
symbol = 'SOL-USDT'
initial_budget = 10.00

bot = TradingBot(initial_budget, symbol)
bot.run()
